## Apartado 1

En este primer ejercicio se implementa la función *string addPunctuationBasic(string)* que, dado un string de entrada que contiene palabras en minísculas y sin signos de puntuación, devuelve dicho string con la primera letra en mayúsculas y un punto al final. Para mayor generalidad, consideramos que la frase pueda contener espacios o tabulaciones al final, y si ya contiene un punto final no añadimos otro.


In [20]:
def addPunctuationBasic(sent):
    sent = sent[0].upper() + sent[1:]
    while sent[-1] == ' ' or sent[-1] == '\t' or sent[-1] == '\n':
        sent = sent[0:-1]
    if sent[-1] == '.' or sent[-1] == ',' or sent[-1] == '.' or sent[-1] == ':' or sent[-1] == ';' or sent[-1] == '?' or sent[-1] == '!':
        return sent
    else:
        return sent + '.'

addPunctuationBasic("i'm Francisco Javier Gañán")

"I'm Francisco Javier Gañán."

## Apartado 2

En este segundo apartado se implementa la función *[(pos, err)] verifyPunctuation(string test, string check)*. Esta función recibe como entrada dos strings y, basándose en la distancia de Levenshtein y en recorrer la matriz que se genera, calcula una lista de mínimas ediciones para convertir *test* en *check*. Se puede encontrar información sobre este algoritmo para calcular la distancia de Levenshtein y la matriz asociada en https://sites.google.com/site/algoritmossimilaridaddistancia/distancia-de-levenshtein. 

En el modelo original de Levenshtein, que es el se usa en este apartado, cada operación de edición (inserción, sustitución o eliminación) tiene coste 1. Por tanto, la distancia de Levenshtein es la suma del número de ediciones realizadas. Needleman y Wunsch, en 1970, lo modificaron para permitir operaciones de edición con distinto costo. Sin embargo, esto no se considera en este ejercicio, ya que, en general, debería asignarse un costo personalizado a cada pareja de palabras para sustitución, eliminación e inserción.

### Tokenizar

La siguiente función (*Tokenize*) se utiliza para tokenizar. Funciona tanto para frases individuales como para textos completos con numerosas frases. Se ha diseñado así para poder tokenizar correctamente los corpus utilizados en este ejercicio.

In [21]:
def Tokenize(seq):
        seqToken = [] # Guarda la secuencia tokenizada
        ant = 0 # Guarda la posición del carácter desde el que comienza un nuevo token
        for i in range(len(seq)):
            if seq[i] == ' ' and not (seq[i-1] == ',' or seq[i-1] == '.' or seq[i-1] == ':' or seq[i-1] == ';' or seq[i-1] == '?' or seq[i-1] == '!' or seq[i-1] == ' ' or seq[i-1] == '\n'):
                seqToken.append(seq[ant:i])
                ant = i + 1
            elif seq[i] == ' ':
                ant = i + 1
            elif seq[i] == ',' or seq[i] == '.' or seq[i] == ':' or seq[i] == ';' or seq[i] == '?' or seq[i] == '!':
                if ant < i:
                    seqToken.append(seq[ant:i])
                    seqToken.append(seq[i])
                    ant = i + 1
                else:
                    seqToken.append(seq[i])
                    ant = i + 1
            elif i == len(seq) - 1 and seq[i] != '\n':
                seqToken.append(seq[ant:i+1])

            # Para leer textos completos considerando el salto de línea. El salto de línea no se considera como token.
            # También se considera que el salto de línea se situa después de un signo de puntuación o de un espacio.
            if seq[i] == "\n":
                ant = i + 1
                
        return seqToken

### Verificación de la puntuación

En la función *verifyPunctuation* se incluyen 3 funciones, que se ejecutan de forma consecutiva:

- **Tokenize**: La presentada en la celda anterior, para tokenizar los *strings* de entrada.

- **matrizLevenshtein**: Calcula la matriz cuyo último elemento (elemento de coordenadas la última fila y la última columna) es la distancia de Levenshtein. Esta matriz se utiliza para calcular posteriormente la lista de ediciones. 

- **ediciones**: Calcula la lista de ediciones recorriendo la matriz generada previamente, llamda **M**. **M** se recorre empezando en el último elemento. En cada iteración, mientras *i* y *j* (índices de la matriz de filas y columnas, respectivamente) no sean ambos iguales a 0:
    1. Se mira en la vecindad, donde *vecinos* = [**M**(*i -1*, *j*), **M**(*i*, *j - 1*), **M**(*i - 1*, *j - 1*)].
    2. Si el valor de algún vecino es igual a **M**(*i*,*j*), se actualizan *j = j - 1* y *i = i - 1*.
       Si no, se selecciona el valor mínimo de todos los vecinos, y se añade a la lista de ediciones la operación correspondiente (en función de las coordenadas del vecino de valor mínimo):

       - ( *i -1*, *j* ) -> ( 'D',*i* ): Se añade una eliminación a la lista de ediciones. Dicha eliminación se incluye en la lista junto con el índice *i*, que se corresponde con el *string* de *check* en este trabajo. 
       - ( *i*, *j - 1* ) -> ( 'I',*i* ): Se añade una inserción. 
       - ( *i -1*, *j - 1* ) -> ( 'S',*i* ): Se añade una sustitución.

       
       Finalmente, se actualizan *i* y *j* con las coordenadas del vecino seleccionado.

Es importante señalar que este algoritmo genera una de las posibles listas de edición que se pueden conseguir, aunque siempre una de distancia mínima. Esto es porque, al recorrer la matriz, si varios vecinos tienen el mismo valor en el paso 2, se ofrece da la posibilidad de escoger varias secuencias de distancia mínima. El algoritmo escogido para este ejercicio da siempre la misma lista de ediciones para dos secuencias iguales.
        

Inspiración para **matrizLevenshtein**:
https://blog.paperspace.com/implementing-levenshtein-distance-word-autocomplete-autocorrect/

Inspiración para **ediciones**:
https://gist.github.com/curzona/9435822

In [22]:
import numpy as np
def verifyPunctuation(test, check):
    ## Primero se tokenizan las dos secuencias mediante la función Tokenize
    checkToken = Tokenize(check)
    testToken = Tokenize(test)

            
    ## A continuación, se calcula la matriz que computa la distancia de Levenshtein
    def matrizLevenshtein(token1, token2):
        matrix = np.zeros((len(token1) + 1, len(token2) + 1)) # Se crea la matriz
        
        for t1 in range(len(token1) + 1): # La primera fila contiene los índices de las posiciones del token 1
            matrix[t1][0] = t1
        
        for t2 in range(len(token2) + 1): # La primera columna contiene los índices de las posiciones del token 2
            matrix[0][t2] = t2
        
        for t1 in range(1, len(token1) + 1):
            for t2 in range(1, len(token2) + 1):
                # Calcuate 
                if (token1[t1-1] == token2[t2-1]):
                    matrix[t1][t2] = matrix[t1 - 1][t2 - 1]
                else:
                    a = matrix[t1][t2 - 1]
                    b = matrix[t1 - 1][t2]
                    c = matrix[t1 - 1][t2 - 1]

                    if (a <= b and a <= c):
                        matrix[t1][t2] = a + 1
                    elif (b <= a and b <= c):
                        matrix[t1][t2] = b + 1
                    else:
                        matrix[t1][t2] = c + 1 

        return matrix
    
    matrix_L = matrizLevenshtein(checkToken,testToken)
    
    ## Se calcula la lista de ediciones
    def ediciones(token1, token2, mat):
        i,j = len(token1), len(token2)
        ediciones = []

        while(not (i==0 and j==0)):
            p = mat[i][j]
            vecinos = []

            if (i!=0 and j!=0):
                vecinos.append(mat[i-1][j-1])

            if (i!=0):
                vecinos.append(mat[i-1][j])
            
            if (j!=0):
                vecinos.append(mat[i][j-1])
            
            min_c = min(vecinos)

            if(min_c == p): # No se añadie ninguna edición
                i, j = i-1, j-1
            elif(j!=0 and min_c == mat[i][j-1]): # Se priorizan las inserciones frente a las sustituciones en el orden para tomar los caminos
                i, j = i, j-1
                ediciones.append(('I', i))
            elif(i!=0 and j!=0 and min_c == mat[i-1][j-1]): # Se priorizan las sustituciones frente a las eliminaciones para tomar los caminos
                i, j = i-1, j-1
                ediciones.append(('S', i))
            elif(i!=0 and min_c == mat[i-1][j]): # Se le da menor prioridad a la hora de escoger los caminos
                i, j = i-1, j
                ediciones.append(('D', i))

        ediciones.reverse() # Como se ha recorrido la matriz a la inversa, se debe invertir la lista para que esté en el orden adecuado
        return ediciones

 
    return ediciones(checkToken, testToken, matrix_L) #, len(testToken), len(checkToken)

A continuación, se comprueba que la función genera una lista de ediciones adecuada.

In [23]:
edit = verifyPunctuation("and if so how do you explain it", "And, if so, how, do you explain it?")
print(edit)

edit = verifyPunctuation("and if so how do you explain it", "And? If so how do you explain it.")
print(edit)

edit = verifyPunctuation("And? If so how do you explain it.", "And, if so, how, do you explain it?")
print(edit)

[('D', 0), ('S', 1), ('D', 4), ('D', 6), ('D', 11)]
[('D', 0), ('S', 1), ('S', 2), ('D', 9)]
[('S', 1), ('S', 2), ('D', 4), ('D', 6), ('S', 11)]


## Apartado 3
 En este apartado se implementa una función (*calculaMetricas*) que calcula el rendimiento de un algoritmo de puntuación, evaluando su resultado en un corpus sobre el que se itera frase a frase. Las métricas que genera son Precisión, *Recall* y F1. También se prueba dicha función para el algoritmo básico de puntuación del apartado 1 sobre el corpus de test.

Lo primero que se hace es leer los corpus de *test* y *check* línea a línea.

In [24]:
## Guardamos los textos de test y check por líneas
f_test = open("datasets/PunctuationTask.test.en","r")
f_check = open("datasets/PunctuationTask.check.en","r")
linesTest = f_test.readlines()
linesCheck = f_check.readlines()
f_test.close()
f_check.close()

A continuación, se calculan las métricas correspondientes. Para ello, es necesario definir los conceptos de TP (*True Positives*), FP (*False Positives*) y FN (*False Negatives*) para un modelo de puntuación. Definimos:

-- **TP**: Cambios correctos que hace el modelo. Estos pueden ser, en general, para los modelos de puntuación que consideramos: Poner mayúscula, poner punto, poner coma, poner dos puntos, poner punto y coma, poner interrogación, poner exclamación. Es decir, añade el signo de puntuación correcto o la mayúscula donde debía realizarse dicho cambio.

-- **FP**: Cambios incorrectos que hace el modelo cuando no se debía hacer un cambio.

-- **FN**: Cambios que omite el modelo que deberían haberse realizado.

**NOTA**: Es importante tener en cuenta que, con estas definiciones, si el modelo realiza un cambio erróneo donde había que realizar un cambio, este cambio no se categoriza como **FP**, ni como **TP**. Esto parece un buen criterio, ya que no afecta tan negativamente al resultado del modelo como un **FP** (pues no empeora la distancia de Levenshtein). 

Por ejemplo: Si el *string* check es "*Qué buen día hace!*" y el *string* test es "qué buen día hace", realizar el cambio "qué buen día hace." no se considera como un **TP**, pues no ha realizado el cambio correcto. Sin embargo, tampoco se considera un **FP**, pues debía realizarse un cambio en esa posición. Contrariamente, si el cambio realizado fuese "qué buen día. hace", este supondría un **FP**, ya que se ha insertado un cambio incorrecto en una posición en la que no se debía realizar ningún cambio.

Para calcular los TP, FP, FN, se sigue el razonamiento ilustrado en la figura.

Si se observan los bloques verdes, estos se corresponden *strings*: con la frase de test original, la frase de check, y la frase que se obtiene al aplicarle el modelo de puntuación. Los bloques azules se corresponden con la distancia de Levenstein entre pares de *strings*. 

Los bloques *verifyPunctuation()* reciben como entrada dos *strings* y devuelven la lista de edición entre ambos. Los bloques *len()* devuelven la longitud de la lista que reciben como entrada, y el bloque *model()* genera los signos de puntuación sobre el *string* de entrada.

- Si se compara el *string* de test con el *string* de Check, se obtiene el número de cambios totales que debe hacer el modelo, esto es **Db = TP + FN**. 

- Si se compara el *string* de test con el *string* del modelo, se obtiene el número de cambios totales que hace el modelo, esto es **Dm = TP + FP**. 

- Si se compara el *string* del modelo con el *string* de Check, ocurre que:
  - **Dcheck** será el resultado de sumar: 
    1. Los **FN**, pues son los cambios que el modelo no ha detectado, cuyas diferencias se siguen manifestado entre ambos *strings*.
    2. Los **FP**, pues son los cambios que el modelo ha hecho indebidamente, que empeoran la distancia de Levenshtein.

**TP**, **FP** y **FN** son incógnitas y **Dm**, **Db** y **Dcheck** son conocidas, por lo que se tiene un sistema de ecuaciones de tres incógnitas compatible determinado que permite conocer dichas incógnitas.

**NOTA**: Es importante destacar que, si se conoce el número de cambios que realiza el modelo, **Dm**, no es necesatio generar dicho número por comparación de strings. Sin embargo, para hacer esta función de evaluación válida para modelos tipo caja negra, se ha decidido computar **Dm** de esta manera.

![Diagrama explicativo](Diagrama_PLN.drawio.png)

In [92]:
## Función para calcular las métricas
def calculaMetricas(line_test, line_check, model):
    ## Iniciaizamos los valores de TP, FP, FN
    TP = 0
    TP_FP = 0
    TP_FN = 0

    # Aplicamos el modelo de puntuacion
    modificado = model(line_test)
    print(line_test)
    print(modificado)
    print(line_check)
    # Calculamos la distancia test-check
    cambios_necesarios = verifyPunctuation(line_test,line_check)
    cambios_modelo = verifyPunctuation(line_test,modificado)
    cambios_finales = verifyPunctuation(modificado,line_check)

    TP_FN = len(cambios_necesarios) # TP + FN
    TP_FP = len(cambios_modelo) # TP + FP

    # Se calculan los TP como las posiciones en las que aparecian cambios 
    # en cambios_necesarios y no se incluyen en cambios finales
    pos_cn = [cn[1] for cn in cambios_necesarios]
    pos_cf = [cf[1] for cf in cambios_finales]
    TP = sum([1 for c in pos_cn if c not in pos_cf])

    FN = TP_FN -TP
    FP = TP_FP -TP
    return TP, FN, FP

Finalmente, mediante la función anterior, se calcula la precisión del modelo básico sobre el conjunto de test.

In [ ]:
# P = 0 
# R = 0
# F1 = 0

# for (line1, line2) in zip(linesTest,linesCheck):
#     evaluation = calculaMetricas(line1, line2, addPunctuationBasic)
#     P += evaluation[0]
#     R += evaluation[1]
#     F1 += evaluation[2]

# P /= len(linesTest)
# R /= len(linesTest)
# F1 /= len(linesTest)

# print("Precision: " + str(P))
# print("Recall: " + str(R))
# print("F1: " + str(F1))

In [75]:
TP = 0 
FN = 0
FP = 0

for (line1, line2) in zip(linesTest,linesCheck):
    evaluation = calculaMetricas(line1, line2, addPunctuationBasic)
    TP += evaluation[0]
    FN += evaluation[1]
    FP += evaluation[2]

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * (P * R) / (P + R)

print("Precision: " + str(P))
print("Recall: " + str(R))
print("F1: " + str(F1))

Precision: 0.9582752855948732
Recall: 0.4451815416477898
F1: 0.6079367183702329


## Apartado 4

En este apartado se realizan dos tareas: 

- Se crea un modelo de puntuación basado en pseudo-cuatrogramas: Este modelo decidirá, en función de las tres palabras anteriores, si realizar una de las siguientes operaciones: Poner la palabra en mayúscula, poner la palabra en minúscula o añadir un signo de puntuación.

- Una función que, utilizando dicho modelo, realice la operación adecuada.


El modelo se genera a partir de un texto tokenizado, realizando los siguientes pasos:

1. Se crean cuatrogramas de la siguiente forma: (**token1**, **token2**, **token3**, **operación**). Esto se hace recorriendo el texto tokenizado de cuatro en cuatro y, para el último token, sustituir el contenido por la 

In [34]:
def pseudoCuatroGramas(Token):
    # Creamos una lista con los pseudo 4-gramas
    cuatroGramas = []
    for i in range(4, len(Token)):
        c_g = Token[(i - 4):i]   
        if c_g[-1][0].islower():
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 0])
        elif c_g[-1][0].isupper():
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 1])
        elif c_g[-1][0] == '.':
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 2])
        elif c_g[-1][0] == ',':
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 3])
        elif c_g[-1][0] == ':':
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 4])
        elif c_g[-1][0] == ';':
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 5])
        elif c_g[-1][0] == '?':
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 6])
        elif c_g[-1][0] == '!':
            cuatroGramas.append([tuple(Token[i - 4:i-1]), 7])
    print(cuatroGramas[:200])
    return cuatroGramas
    

In [ ]:
f_train = open("datasets/PunctuationTask.train.en","r")
linesTrain = f_train.read() # Guardamos el conjunto de entrenamiento
f_train.close()
trainToken = Tokenize(linesTrain)
cuatroGramas = pseudoCuatroGramas(trainToken)

In [64]:
# Se crea un diccionario para almacenar los tokens únicos y su valor
fDist = dict()
for cg in cuatroGramas:
    if tuple(cg) in fDist:
        fDist[tuple(cg)] += 1
    else:
        fDist[tuple(cg)] = 1
    

In [90]:
## Se crea la función que añade los signos de puntuación

def addPunctuation4gram(sent, addMayusc = True, addDot = True):
    sentToken = Tokenize(sent)

    # Para añadir la primera palabra en mayúscula
    if addMayusc:
        sentToken[0] = sentToken[0].capitalize()

    i = 0
    while i < len(sentToken) - 2:
        valor = tuple(sentToken[i:i + 3])
        max = 0
        max_index = 0
        flag = False
        for j in range(8):
            if tuple([valor, j]) in fDist:
                flag = True
                act = fDist[tuple([valor, j])]
            else:
                continue
            if act > max:
                max = act
                max_index = j
        
        if flag == True:
            # Dependiendo del valor máximo, se eliige una acción u otra
            if max_index == 0 and (i + 3) < len(sentToken) and not sentToken[i+3].islower():
                sentToken[i+3] = sentToken[i+3][0].lower() + sentToken[i+3][1:]
            if max_index == 1 and (i + 3) < len(sentToken):
                sentToken[i+3] = sentToken[i+3].capitalize() 
            elif max_index == 2:
                sentToken.insert(i+3, '.') 
            elif max_index == 3:
                sentToken.insert(i+3, ',') 
            elif max_index == 4:
                sentToken.insert(i+3, ':')    
            elif max_index == 5:
                sentToken.insert(i+3, ';') 
            elif max_index == 6:
                sentToken.insert(i+3, '?') 
            elif max_index == 7:
                sentToken.insert(i+3, '!')
            elif addDot and (i + 3) == len(sentToken) and sentToken[i+2] not in ['.', '?', '!']:
                sentToken.insert(i+3, '.') 
        elif addDot and (i + 3) == len(sentToken) and sentToken[i+2] not in ['.', '?', '!']:
            sentToken.insert(i+3, '.')

        i += 1
    sentReturn = ''
    for k, i in enumerate(sentToken):
        if i == ',' or i == '.' or i == ':' or i == ';' or i == '?' or i == '!' or k == 0:
            sentReturn = sentReturn + i
        else:
            sentReturn = sentReturn + ' ' + i
    return sentReturn

## Apartado 5

In [ ]:
# P = 0 
# R = 0
# F1 = 0

# for (line1, line2) in zip(linesTest[:10],linesCheck[:10]):
#     evaluation = calculaMetricas(line1, line2, addPunctuation4gram)
#     P += evaluation[0]
#     R += evaluation[1]
#     F1 += evaluation[2]

# P /= len(linesTest)
# R /= len(linesTest)
# F1 /= len(linesTest)

# print("Precision: " + str(P))
# print("Recall: " + str(R))
# print("F1: " + str(F1))

In [93]:
TP = 0 
FN = 0
FP = 0

for (line1, line2) in zip(linesTest[:10],linesCheck[:10]):
    evaluation = calculaMetricas(line1, line2, addPunctuation4gram)
    TP += evaluation[0]
    FN += evaluation[1]
    FP += evaluation[2]

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * (P * R) / (P + R)

print("Precision: " + str(P))
print("Recall: " + str(R))
print("F1: " + str(F1))

it can be a very complicated thing the ocean 

It can be a very complicated thing, the ocean.
It can be a very complicated thing, the ocean. 

and we're making the ocean pretty unhappy in a lot of different ways 

And we're making the ocean pretty unhappy in a lot of different ways.
And we're making the ocean pretty unhappy in a lot of different ways. 

people working in these canneries could barely stay there all day because of the smell but you know what they came out saying 

People working in these canneries could barely stay there all day because of the smell. But you know, what they came out saying.
People working in these canneries could barely stay there all day because of the smell, but you know what they came out saying? 

we made the ocean unhappy we made people very unhappy and we made them unhealthy 

We made the ocean unhappy we made people very unhappy and we made them unhealthy.
We made the ocean unhappy; we made people very unhappy, and we made them unhealthy. 

we see

In [167]:
# def mix_model(sent):
#     sent2 = addPunctuation4gram(sent)
#     return addPunctuationBasic(sent2)

In [ ]:
# P = 0 
# R = 0
# F1 = 0

# for (line1, line2) in zip(linesTest,linesCheck):
#     evaluation = calculaMetricas(line1, line2, mix_model)
#     P += evaluation[0]
#     R += evaluation[1]
#     F1 += evaluation[2]

# P /= len(linesTest)
# R /= len(linesTest)
# F1 /= len(linesTest)

# print("Precision: " + str(P))
# print("Recall: " + str(R))
# print("F1: " + str(F1))

## Apartado 6

In [6]:
#cambiamos los textos para que puedan ser leídos por el modelo
f_test = open("datasets/PunctuationTask.test.en","r")
f_check = open("datasets/PunctuationTask.check.en","r")
f_train = open("datasets/PunctuationTask.train.en","r")
Test = f_test.read()
Check = f_check.read()
Train = f_train.read()
f_test.close()
f_check.close()
f_train.close()

Check_new = ""
for c in Check:
    if c == ',' or c == '.' or c == ':' or c == ';' or c == '?' or c == '!':
        Check_new += " "
    Check_new += c

Train_new = ""
for c in Train:
    if c == ',' or c == '.' or c == ':' or c == ';' or c == '?' or c == '!':
        Train_new += " "
    Train_new += c


with open('datasets/test.txt', 'w') as f:
    f.write(Test.lower())
f.close()
with open('datasets/dev.txt', 'w') as f:
    f.write(Check_new.lower())
f.close()
with open('datasets/train.txt', 'w') as f:
    f.write(Train_new.lower())
f.close()